In [3]:
pip install pyomo

Note: you may need to restart the kernel to use updated packages.


In [4]:
# from collections import defaultdict
from pyomo.util.infeasible import log_infeasible_constraints

from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd
import numpy as np
import datetime

In [5]:
pd.set_option('display.max_columns', 20)

In [6]:
def remove_nan(dic):  # remove nan from a dictionary
    for k in dic.keys():
        for v in range(len(dic[k])):
            try:
                if np.isnan(dic[k][v]):
                    del dic[k][v]
            except:
                pass
    return dic


## Employee Requirement For Assignment

In [7]:
# Employee Requirement for Assignment in each TSlot
TimeAssEmpReq = pd.read_csv('/home/ZONE24X7-CMB/hasalf/FMSI/Data/Day_01/Time_Ass_Req_Schedule.csv', index_col=False)

#TimeAssEmpReq['Time'] = TimeAssEmpReq['Time'].str.split(' ').str[0].replace(':', '.', regex=True).str.strip("0")
TimeAssEmpReq['Time'] = TimeAssEmpReq['Time'].str.split(' ').str[0]
#timeDF=(pd.to_timedelta(TimeAssEmpReq['Time'].str.strip()))
TimeAssEmpReq['Time']=(pd.to_datetime(TimeAssEmpReq['Time'].str.strip(), format='%H:%M:%S')).dt.time

TimeAssEmpReq.head()


,Day,Time,Assignment,Requirement,Scheduled
0,15/11/2018,09:00:00,Downtown Lobby,3.8,2
1,15/11/2018,09:15:00,Downtown Lobby,3.2,2
2,15/11/2018,09:30:00,Downtown Lobby,2.6,2
3,15/11/2018,09:45:00,Downtown Lobby,2.8,2
4,15/11/2018,10:00:00,Downtown Lobby,4.1,2


In [8]:
# define Time Slots
TSlots = TimeAssEmpReq.Time.unique()
print(TSlots)

[datetime.time(9, 0) datetime.time(9, 15) datetime.time(9, 30)
 datetime.time(9, 45) datetime.time(10, 0) datetime.time(10, 15)
 datetime.time(10, 30) datetime.time(10, 45) datetime.time(11, 0)
 datetime.time(11, 15) datetime.time(11, 30) datetime.time(11, 45)
 datetime.time(12, 0) datetime.time(12, 15) datetime.time(12, 30)
 datetime.time(12, 45) datetime.time(1, 0) datetime.time(1, 15)
 datetime.time(1, 30) datetime.time(1, 45) datetime.time(2, 0)
 datetime.time(2, 15) datetime.time(2, 30) datetime.time(2, 45)
 datetime.time(3, 0) datetime.time(3, 15) datetime.time(3, 30)
 datetime.time(3, 45) datetime.time(4, 0) datetime.time(4, 15)
 datetime.time(4, 30) datetime.time(4, 45) datetime.time(8, 30)
 datetime.time(8, 45) datetime.time(5, 0) datetime.time(5, 15)]


## Employee Details

In [9]:
# define employees
EmpData = pd.read_csv('/home/ZONE24X7-CMB/hasalf/FMSI/Data/Day_01/EmpTasks.csv', index_col=False)
EmpData.head()

,Day,Employee,Task
0,15/11/2018,Employee501603,Downtown Platform
1,15/11/2018,Employee501603,MSR Opener Platform
2,15/11/2018,Employee501603,MSR Closer Platform
3,15/11/2018,Employee501715,Downtown Lobby
4,15/11/2018,Employee501715,MSR Opener Lobby


In [10]:
Employees = EmpData.Employee.unique()
print(Employees)

['Employee501603' 'Employee501715' 'Employee501716' 'Employee501721'
 'Employee501768' 'Employee501792' 'Employee501587' 'Employee501595']


In [11]:
# Assignments
Assignments = TimeAssEmpReq.Assignment.unique()

In [12]:
AssTslots = TimeAssEmpReq.groupby('Time')['Assignment'].apply(list).to_dict()
Ass_Tslots = remove_nan(AssTslots)
print(AssTslots)

{datetime.time(1, 0): ['Downtown Lobby', 'Downtown Platform', 'BM Admin Time', 'ABM Admin Time'], datetime.time(1, 15): ['Downtown Lobby', 'Downtown Platform', 'BM Admin Time', 'ABM Admin Time'], datetime.time(1, 30): ['Downtown Lobby', 'Downtown Platform', 'BM Admin Time', 'ABM Admin Time'], datetime.time(1, 45): ['Downtown Lobby', 'Downtown Platform', 'BM Admin Time', 'ABM Admin Time'], datetime.time(2, 0): ['Downtown Lobby', 'Downtown Platform', 'BM Admin Time', 'ABM Admin Time'], datetime.time(2, 15): ['Downtown Lobby', 'Downtown Platform', 'BM Admin Time', 'ABM Admin Time'], datetime.time(2, 30): ['Downtown Lobby', 'Downtown Platform', 'BM Admin Time', 'ABM Admin Time'], datetime.time(2, 45): ['Downtown Lobby', 'Downtown Platform', 'BM Admin Time', 'ABM Admin Time'], datetime.time(3, 0): ['Downtown Lobby', 'Downtown Platform', 'BM Admin Time', 'ABM Admin Time'], datetime.time(3, 15): ['Downtown Lobby', 'Downtown Platform', 'BM Admin Time', 'ABM Admin Time'], datetime.time(3, 30): 

In [13]:
# Employee Eligibility for each assignments
Emp_Elig = EmpData.groupby('Employee')['Task'].apply(list).to_dict()
Emp_Elig = remove_nan(Emp_Elig)
print(Emp_Elig)


{'Employee501587': ['Downtown Platform', 'Downtown Lobby'], 'Employee501595': ['Downtown Platform', 'MSR Opener Platform', 'MSR Closer Platform'], 'Employee501603': ['Downtown Platform', 'MSR Opener Platform', 'MSR Closer Platform'], 'Employee501715': ['Downtown Lobby', 'MSR Opener Lobby', 'MSR Closer Lobby'], 'Employee501716': ['Downtown Lobby', 'Teller Closer Lobby'], 'Employee501721': ['Downtown Platform', 'MSR Opener Platform', 'MSR Closer Platform', 'BM Admin Time'], 'Employee501768': ['Downtown Lobby', 'Teller Closer Lobby'], 'Employee501792': ['Downtown Platform', 'MSR Opener Platform', 'MSR Closer Platform']}


In [14]:
EmpWorkHours = pd.read_csv('/home/ZONE24X7-CMB/hasalf/FMSI/Data/Day_01/EmpWorkHours.csv', index_col=False)
EmpMinWork = pd.Series(EmpWorkHours.Min_Work.values, index=EmpWorkHours.Employee).to_dict()
EmpMaxWork = pd.Series(EmpWorkHours.Max_Work.values, index=EmpWorkHours.Employee).to_dict()
print(EmpMaxWork)

{'Employee501603': 8, 'Employee501715': 8, 'Employee501716': 8, 'Employee501721': 8, 'Employee501768': 8, 'Employee501792': 8, 'Employee501587': 8, 'Employee501595': 8}


In [15]:
# Employee Requirement
Emp_Req = TimeAssEmpReq.groupby('Time')[['Assignment', 'Requirement']] \
    .apply(lambda x: pd.Series(x.Requirement.values, index=x.Assignment).to_dict()).to_dict()

In [16]:
# convert Emp_Req dictionaries values in to Lists
l = []
for k in Emp_Req.keys():
    a = list(Emp_Req[k].values())
    l.append(a)

Emp_requiremnt = sum(sum(x) for x in l)  # calculate employee requirement

No_of_Assignments = sum([len(x) for x in Ass_Tslots.values()])

# Model Creation

In [17]:
# Initialize model
model = ConcreteModel()

# variable to represent employee allocation to assignments (decision variable)
model.works = Var(
    (((Employee, TSlot, Assignment) for Employee in Employees for TSlot in TSlots for Assignment in Ass_Tslots[TSlot])),
    within=Binary, initialize=0)

model.MI = Var(bounds=(0, 5))
model.MA = Var(bounds=(0, 5))


In [18]:
def getPreviousTimeslot(TSLOT):
    return (datetime.datetime.combine(datetime.date(1, 1, 1), TSLOT) - datetime.timedelta(minutes=15)).time()


### Objective Function

In [19]:
# function to define Objective function - original
# def obj_rule(m):
#     obj = (Emp_requiremnt - sum(
#         m.works[Employee, TSlot, Assignment] for Employee in Employees for TSlot in TSlots for Assignment in
#         Ass_Tslots[TSlot]) + m.MI)
#     return obj

In [20]:
# function to define Objective function - Minimize idle times between allocations
def obj_rule(m):
    obj = (Emp_requiremnt - sum(m.works[Employee, TSlot, Assignment] 
                   for Employee in Employees 
                   for TSlot in TSlots 
                   for Assignment in Ass_Tslots[TSlot])
          ) 
    + m.MI 
    + sum((max((value(m.works[Employee, TSlot, Assignment]) - value(m.works[(Employee, getPreviousTimeslot(TSlot), Assignment)])) , 
               (value(m.works[(Employee, getPreviousTimeslot(TSlot), Assignment)]) - value(m.works[Employee, TSlot, Assignment]) ) )) 
          for Employee in Employees 
          for TSlot in TSlots[1:] 
          for Assignment in Ass_Tslots[TSlot] 
          if (Employee,getPreviousTimeslot(TSlot), Assignment) in m.works)
    
    return obj

In [21]:
# Add Objective function to the model
model.obj = Objective(rule=obj_rule, sense=minimize)

### Constraints

In [22]:
# Model constraints
model.constraints = ConstraintList()  # Create a Set of Constraints

#### 1. Employee Allocation <= Requirement

In [23]:
# Maximum Employee Requirement
for TSlot in TSlots:
    for Assignment in Ass_Tslots[TSlot]:
        model.constraints.add(
            Emp_Req[TSlot][Assignment] >= sum(model.works[Employee, TSlot, Assignment] for Employee in Employees)
        )

#### 2. Daily Max work hours

In [24]:
# Max Work Hours
for Employee in Employees:
    model.constraints.add(
        EmpMaxWork[Employee] >= 0.25 * sum(
            model.works[Employee, TSlot, Assignment] for TSlot in TSlots for Assignment in Ass_Tslots[TSlot])
    )

#### 3. Daily Min work hours

In [25]:
# Min Work Hours
for Employee in Employees:
    model.constraints.add(
        EmpMinWork[Employee] <= 0.25 * sum(
            model.works[Employee, TSlot, Assignment] for TSlot in TSlots for Assignment in Ass_Tslots[TSlot]) + model.MI
    )

#### 4. No overlapping assignments for employees

In [26]:
# No allocation for overlapping Assignments
for Employee in Employees:
    for TSlot in TSlots:
        model.constraints.add(
            1 >= sum(model.works[Employee, TSlot, Assignment] for Assignment in Ass_Tslots[TSlot])
        )

#### 5. Employee eligibility to task

In [27]:
# Employee Eligibility to Allocate to Assignments (remove this with decision variable)
for Employee in Employees:
    for TSlot in Ass_Tslots:
        for Assignment in list(set(Ass_Tslots[TSlot]).intersection(set(Assignments) - set(Emp_Elig[Employee]))):
            model.constraints.add(
                0 == model.works[Employee, TSlot, Assignment]
            )

#### 6. Weekly Min work hours

#### 7. Weekly max work hours

In [28]:
# Constraint - An employee can't be allocated to one task more than once within the day
# for Employee in Employees:
#     for Assignment in Assignments: 
#         model.constraints.add(
#             1 >= sum(model.works[Employee, TSlot, Assignment] - model.works[Employee, getPreviousTimeslot(TSlot), Assignment] 
#               for TSlot in TSlots[1:] if ((Employee,getPreviousTimeslot(TSlot), Assignment) in model.works and (Employee,TSlot, Assignment) in model.works ))
#         )


In [29]:
# Constraint - An employee must work in one section atleast for an hour
#TODO: Should be able to take the minimum hours to work in a section as an input



In [30]:
opt = SolverFactory('cbc')
results = opt.solve(model, tee=True)
log_infeasible_constraints(model)  #Get infeasible constraints

Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Aug 21 2017 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmp5gxmglts.pyomo.lp -stat=1 -solve -solu /tmp/tmp5gxmglts.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 120 (-1105) rows, 340 (-782) columns and 696 (-4573) elements
Statistics for presolved model
Original problem has 1120 integers (1120 of which binary)
Presolved problem has 340 integers (340 of which binary)
==== 0 zero objective 1 different
340 variables have objective of -1
==== absolute objective values 1 different
340 variables have objective of 1
==== for integers 0 zero objective 1 different
340 variables have objective of -1
==== for integers absolute objective values 1 different
340 variables have objective of 1
===== end objective counts


Problem has 120 rows, 340 columns (340 with objective) and 696 elements
There are 58 singletons with objective 
Column breakdown:
0 of type 0.0->inf, 0 of 

In [31]:
model.pprint()

2 Set Declarations
    constraints_index : Dim=0, Dimen=1, Size=1224, Domain=None, Ordered=False, Bounds=None
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198,

                ('Employee501603', datetime.time(12, 45), 'BM Admin Time') :     0 :   0.0 :     1 : False : False : Binary
               ('Employee501603', datetime.time(12, 45), 'Downtown Lobby') :     0 :   0.0 :     1 : False : False : Binary
            ('Employee501603', datetime.time(12, 45), 'Downtown Platform') :     0 :   1.0 :     1 : False : False : Binary
                 ('Employee501715', datetime.time(1, 0), 'ABM Admin Time') :     0 :   0.0 :     1 : False : False : Binary
                  ('Employee501715', datetime.time(1, 0), 'BM Admin Time') :     0 :   0.0 :     1 : False : False : Binary
                 ('Employee501715', datetime.time(1, 0), 'Downtown Lobby') :     0 :   1.0 :     1 : False : False : Binary
              ('Employee501715', datetime.time(1, 0), 'Downtown Platform') :     0 :   0.0 :     1 : False : False : Binary
                ('Employee501715', datetime.time(1, 15), 'ABM Admin Time') :     0 :   0.0 :     1 : False : False : Binary
        

                ('Employee501721', datetime.time(3, 15), 'ABM Admin Time') :     0 :   0.0 :     1 : False : False : Binary
                 ('Employee501721', datetime.time(3, 15), 'BM Admin Time') :     0 :   1.0 :     1 : False : False : Binary
                ('Employee501721', datetime.time(3, 15), 'Downtown Lobby') :     0 :   0.0 :     1 : False : False : Binary
             ('Employee501721', datetime.time(3, 15), 'Downtown Platform') :     0 :   0.0 :     1 : False : False : Binary
                ('Employee501721', datetime.time(3, 30), 'ABM Admin Time') :     0 :   0.0 :     1 : False : False : Binary
                 ('Employee501721', datetime.time(3, 30), 'BM Admin Time') :     0 :   1.0 :     1 : False : False : Binary
                ('Employee501721', datetime.time(3, 30), 'Downtown Lobby') :     0 :   0.0 :     1 : False : False : Binary
             ('Employee501721', datetime.time(3, 30), 'Downtown Platform') :     0 :   0.0 :     1 : False : False : Binary
        

             ('Employee501792', datetime.time(1, 30), 'Downtown Platform') :     0 :   1.0 :     1 : False : False : Binary
                ('Employee501792', datetime.time(1, 45), 'ABM Admin Time') :     0 :   0.0 :     1 : False : False : Binary
                 ('Employee501792', datetime.time(1, 45), 'BM Admin Time') :     0 :   0.0 :     1 : False : False : Binary
                ('Employee501792', datetime.time(1, 45), 'Downtown Lobby') :     0 :   0.0 :     1 : False : False : Binary
             ('Employee501792', datetime.time(1, 45), 'Downtown Platform') :     0 :   1.0 :     1 : False : False : Binary
                 ('Employee501792', datetime.time(2, 0), 'ABM Admin Time') :     0 :   0.0 :     1 : False : False : Binary
                  ('Employee501792', datetime.time(2, 0), 'BM Admin Time') :     0 :   0.0 :     1 : False : False : Binary
                 ('Employee501792', datetime.time(2, 0), 'Downtown Lobby') :     0 :   0.0 :     1 : False : False : Binary
        

        Key  : Active : Sense    : Expression
        None :   True : minimize : 302.50000000000006 - (works[Employee501603,09:00:00,Downtown Lobby] + works[Employee501603,09:00:00,Downtown Platform] + works[Employee501603,09:00:00,BM Admin Time] + works[Employee501603,09:00:00,ABM Admin Time] + works[Employee501603,09:15:00,Downtown Lobby] + works[Employee501603,09:15:00,Downtown Platform] + works[Employee501603,09:15:00,BM Admin Time] + works[Employee501603,09:15:00,ABM Admin Time] + works[Employee501603,09:30:00,Downtown Lobby] + works[Employee501603,09:30:00,Downtown Platform] + works[Employee501603,09:30:00,BM Admin Time] + works[Employee501603,09:30:00,ABM Admin Time] + works[Employee501603,09:45:00,Downtown Lobby] + works[Employee501603,09:45:00,Downtown Platform] + works[Employee501603,09:45:00,BM Admin Time] + works[Employee501603,09:45:00,ABM Admin Time] + works[Employee501603,10:00:00,Downtown Lobby] + works[Employee501603,10:00:00,Downtown Platform] + works[Employee501603,1

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [32]:
# Print Employees who assigned to each assignment in all Time Slots

Dic = {TSlot: {Assignment: [] for Assignment in Ass_Tslots[TSlot]} for TSlot in TSlots}
for Employee in Employees:
    for TSlot in TSlots:
        for Assignment in Ass_Tslots[TSlot]:
            if model.works[Employee, TSlot, Assignment] == 1:
                Dic[TSlot][Assignment].append(Employee)

In [33]:
print(Dic)

{datetime.time(9, 0): {'Downtown Lobby': ['Employee501715', 'Employee501768', 'Employee501587'], 'Downtown Platform': ['Employee501792', 'Employee501595'], 'BM Admin Time': ['Employee501721'], 'ABM Admin Time': []}, datetime.time(9, 15): {'Downtown Lobby': ['Employee501715', 'Employee501768', 'Employee501587'], 'Downtown Platform': ['Employee501603', 'Employee501792'], 'BM Admin Time': ['Employee501721'], 'ABM Admin Time': []}, datetime.time(9, 30): {'Downtown Lobby': ['Employee501715', 'Employee501716'], 'Downtown Platform': ['Employee501792', 'Employee501587', 'Employee501595'], 'BM Admin Time': ['Employee501721'], 'ABM Admin Time': []}, datetime.time(9, 45): {'Downtown Lobby': ['Employee501715', 'Employee501768'], 'Downtown Platform': ['Employee501603', 'Employee501792', 'Employee501587', 'Employee501595'], 'BM Admin Time': ['Employee501721'], 'ABM Admin Time': []}, datetime.time(10, 0): {'Downtown Lobby': ['Employee501715', 'Employee501716', 'Employee501768', 'Employee501587'], 'Do

In [34]:
print(pd.DataFrame(Dic))

                                                                09:00:00  \
Downtown Lobby          [Employee501715, Employee501768, Employee501587]   
Downtown Platform                       [Employee501792, Employee501595]   
BM Admin Time                                           [Employee501721]   
ABM Admin Time                                                        []   
BMT Meeting                                                          NaN   
Teller Opener Platform                                               NaN   
MSR Opener Platform                                                  NaN   
Teller Opener Lobby                                                  NaN   
MSR Opener Lobby                                                     NaN   
Teller Closer Platform                                               NaN   
MSR Closer Platform                                                  NaN   
Teller Closer Lobby                                                  NaN   
MSR Closer L

In [35]:
x =pd.DataFrame(Dic)
x.head(10)

,09:00:00,09:15:00,09:30:00,09:45:00,10:00:00,10:15:00,10:30:00,10:45:00,11:00:00,11:15:00,...,03:30:00,03:45:00,04:00:00,04:15:00,04:30:00,04:45:00,08:30:00,08:45:00,05:00:00,05:15:00
Downtown Lobby,"[Employee501715, Employee501768, Employee501587]","[Employee501715, Employee501768, Employee501587]","[Employee501715, Employee501716]","[Employee501715, Employee501768]","[Employee501715, Employee501716, Employee50176...","[Employee501716, Employee501768]","[Employee501715, Employee501768, Employee501587]","[Employee501715, Employee501768, Employee501587]","[Employee501715, Employee501716, Employee50176...","[Employee501715, Employee501716, Employee50176...",...,"[Employee501715, Employee501716, Employee50176...","[Employee501715, Employee501716, Employee50176...","[Employee501715, Employee501768, Employee501587]","[Employee501715, Employee501716, Employee50176...","[Employee501715, Employee501716, Employee50176...","[Employee501715, Employee501716, Employee50176...",NaN,NaN,NaN,NaN
Downtown Platform,"[Employee501792, Employee501595]","[Employee501603, Employee501792]","[Employee501792, Employee501587, Employee501595]","[Employee501603, Employee501792, Employee50158...","[Employee501603, Employee501792, Employee501595]","[Employee501603, Employee501792, Employee501595]","[Employee501603, Employee501595]","[Employee501603, Employee501595]","[Employee501603, Employee501595]","[Employee501603, Employee501792]",...,"[Employee501603, Employee501792, Employee501595]","[Employee501603, Employee501792, Employee501595]","[Employee501603, Employee501792, Employee501595]","[Employee501721, Employee501595]","[Employee501721, Employee501792, Employee501595]","[Employee501603, Employee501721, Employee501792]",NaN,NaN,NaN,NaN
BM Admin Time,[Employee501721],[Employee501721],[Employee501721],[Employee501721],[Employee501721],[Employee501721],[Employee501721],[Employee501721],[Employee501721],[Employee501721],...,[Employee501721],[Employee501721],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABM Admin Time,[],[],[],[],[],[],[],[],[],[],...,[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BMT Meeting,NaN,NaN,NaN,NaN,[],[],[],[],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Teller Opener Platform,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[],[],NaN,NaN
MSR Opener Platform,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[Employee501792],[Employee501603],NaN,NaN
Teller Opener Lobby,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[],[],NaN,NaN
MSR Opener Lobby,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[Employee501715],[Employee501715],NaN,NaN
Teller Closer Platform,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]


In [36]:
#Write the final schedule to a csv file
x.to_csv("schedule1.csv", sep=',')

In [37]:
x.iloc[7,5]

nan

In [38]:
#write the number of allocated employees to a csv file
# Print Employees who assigned to each assignment in all Time SlotsDic = {TSlot: {Assignment: [] for Assignment in Ass_Tslots[TSlot]} for TSlot in TSlots}
Dic2 = {}
for TSlot in TSlots:
    count = 0
    for Assignment in Ass_Tslots[TSlot]:
        for Employee in Employees:
            if model.works[Employee, TSlot, Assignment] == 1:
                count = count + 1
    Dic2[TSlot] = count
            

In [39]:
print(Dic2)

{datetime.time(9, 0): 6, datetime.time(9, 15): 6, datetime.time(9, 30): 6, datetime.time(9, 45): 7, datetime.time(10, 0): 8, datetime.time(10, 15): 6, datetime.time(10, 30): 6, datetime.time(10, 45): 6, datetime.time(11, 0): 7, datetime.time(11, 15): 7, datetime.time(11, 30): 7, datetime.time(11, 45): 6, datetime.time(12, 0): 7, datetime.time(12, 15): 7, datetime.time(12, 30): 6, datetime.time(12, 45): 8, datetime.time(1, 0): 7, datetime.time(1, 15): 8, datetime.time(1, 30): 7, datetime.time(1, 45): 7, datetime.time(2, 0): 6, datetime.time(2, 15): 7, datetime.time(2, 30): 7, datetime.time(2, 45): 7, datetime.time(3, 0): 5, datetime.time(3, 15): 7, datetime.time(3, 30): 8, datetime.time(3, 45): 8, datetime.time(4, 0): 6, datetime.time(4, 15): 6, datetime.time(4, 30): 7, datetime.time(4, 45): 7, datetime.time(8, 30): 2, datetime.time(8, 45): 2, datetime.time(5, 0): 3, datetime.time(5, 15): 3}


In [40]:
from pandas import DataFrame
df = pd.DataFrame(
    [{"Time": time, "Allocation": allocation} for time, allocation in Dic2.items()])

In [41]:
df.head()

,Time,Allocation
0,09:00:00,6
1,09:15:00,6
2,09:30:00,6
3,09:45:00,7
4,10:00:00,8


In [42]:
df.to_csv("allocation1.csv", sep=',')

In [43]:
x1 = x.fillna(0)
x1

,09:00:00,09:15:00,09:30:00,09:45:00,10:00:00,10:15:00,10:30:00,10:45:00,11:00:00,11:15:00,...,03:30:00,03:45:00,04:00:00,04:15:00,04:30:00,04:45:00,08:30:00,08:45:00,05:00:00,05:15:00
Downtown Lobby,"[Employee501715, Employee501768, Employee501587]","[Employee501715, Employee501768, Employee501587]","[Employee501715, Employee501716]","[Employee501715, Employee501768]","[Employee501715, Employee501716, Employee50176...","[Employee501716, Employee501768]","[Employee501715, Employee501768, Employee501587]","[Employee501715, Employee501768, Employee501587]","[Employee501715, Employee501716, Employee50176...","[Employee501715, Employee501716, Employee50176...",...,"[Employee501715, Employee501716, Employee50176...","[Employee501715, Employee501716, Employee50176...","[Employee501715, Employee501768, Employee501587]","[Employee501715, Employee501716, Employee50176...","[Employee501715, Employee501716, Employee50176...","[Employee501715, Employee501716, Employee50176...",0,0,0,0
Downtown Platform,"[Employee501792, Employee501595]","[Employee501603, Employee501792]","[Employee501792, Employee501587, Employee501595]","[Employee501603, Employee501792, Employee50158...","[Employee501603, Employee501792, Employee501595]","[Employee501603, Employee501792, Employee501595]","[Employee501603, Employee501595]","[Employee501603, Employee501595]","[Employee501603, Employee501595]","[Employee501603, Employee501792]",...,"[Employee501603, Employee501792, Employee501595]","[Employee501603, Employee501792, Employee501595]","[Employee501603, Employee501792, Employee501595]","[Employee501721, Employee501595]","[Employee501721, Employee501792, Employee501595]","[Employee501603, Employee501721, Employee501792]",0,0,0,0
BM Admin Time,[Employee501721],[Employee501721],[Employee501721],[Employee501721],[Employee501721],[Employee501721],[Employee501721],[Employee501721],[Employee501721],[Employee501721],...,[Employee501721],[Employee501721],0,0,0,0,0,0,0,0
ABM Admin Time,[],[],[],[],[],[],[],[],[],[],...,[],[],0,0,0,0,0,0,0,0
BMT Meeting,0,0,0,0,[],[],[],[],0,0,...,0,0,0,0,0,0,0,0,0,0
Teller Opener Platform,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,[],[],0,0
MSR Opener Platform,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,[Employee501792],[Employee501603],0,0
Teller Opener Lobby,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,[],[],0,0
MSR Opener Lobby,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,[Employee501715],[Employee501715],0,0
Teller Closer Platform,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[],[]


In [52]:
import csv

x2 = pd.DataFrame()
for index,value in x1.iterrows():
    for col in x1.columns:
        try:
            value[col] = len(value[col])
        except:
            value[col] = 0
    
    x2 = x2.append(value)     

#Write the final schedule to a csv file
x2.to_csv("schedule1counts.csv", sep=',')
x3 = x2.T

downtownLobby = x3['Downtown Lobby']
downtownPlatform = x3['Downtown Platform']
bmAdminTime = x3['BM Admin Time']
abmAdminTime = x3['ABM Admin Time']
bmtMeeting = x3['BMT Meeting']
toPlatform = x3['Teller Opener Platform']
msrOPlatform = x3['MSR Opener Platform']
toLobby = x3['Teller Opener Lobby']
msrOLobby = x3['MSR Opener Lobby']
tellerCloserPlatform = x3['Teller Closer Platform']
msrCloserPlatform = x3['MSR Closer Platform']
tellerCloserLobby = x3['Teller Closer Lobby']
msrCloserLobby = x3['MSR Closer Lobby']

x3
# data_dict = x2.to_dict()
# # print(data_dict)
# # x2.rename_axis('Tasks')
# in_file = pd.read_csv("schedule1counts.csv")
# in_file
# lines = list(in_file)
# lines[0][0] = 'Tasks'
# writer = csv.writer(open('schedule1counts.csv', 'w'))
# writer.writerows(lines)

# cols = x2.columns
# test_list = deque(cols) 
# test_list.appendleft(0) 
# test_list = list(test_list) 
# print(test_list)
# for row in in_file:
#     row = 'Tasks,01:00:00,01:15:00,01:30:00,01:45:00,02:00:00,02:15:00,02:30:00,02:45:00,03:00:00,03:15:00,03:30:00,03:45:00,04:00:00,04:15:00,04:30:00,04:45:00,05:00:00,05:15:00,08:30:00,08:45:00,09:00:00,09:15:00,09:30:00,09:45:00,10:00:00,10:15:00,10:30:00,10:45:00,11:00:00,11:15:00,11:30:00,11:45:00,12:00:00,12:15:00,12:30:00,12:45:00\n'
#     print(row)
#     break
# in_file_df = pd.DataFrame(in_file)
# in_file_df

,Downtown Lobby,Downtown Platform,BM Admin Time,ABM Admin Time,BMT Meeting,Teller Opener Platform,MSR Opener Platform,Teller Opener Lobby,MSR Opener Lobby,Teller Closer Platform,MSR Closer Platform,Teller Closer Lobby,MSR Closer Lobby
01:00:00,3.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01:15:00,4.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01:30:00,4.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01:45:00,4.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02:00:00,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02:15:00,4.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02:30:00,4.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02:45:00,4.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
03:00:00,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
03:15:00,3.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
from collections import OrderedDict
assignmentList = list()
for index, value in TimeAssEmpReq.iterrows():
#    print(value)
    if value['Assignment'] not in assignmentList:
        assignmentList.append(value['Assignment'])
    
    x3.loc[value['Time'], value['Assignment']+' Requirement'] = value['Requirement']
    if value['Assignment']+' Requirement' not in assignmentList:
        assignmentList.append(value['Assignment']+' Requirement')
    
    x3.loc[value['Time'], value['Assignment']+' Scheduled'] = value['Scheduled']
    if value['Assignment']+' Scheduled' not in assignmentList:
        assignmentList.append(value['Assignment']+' Scheduled')
x3 = x3.fillna(0)
x3 = x3.reindex(columns=assignmentList)
x3.to_csv("comparison.csv", sep=',')
x3

,Downtown Lobby,Downtown Lobby Requirement,Downtown Lobby Scheduled,Downtown Platform,Downtown Platform Requirement,Downtown Platform Scheduled,Teller Opener Platform,Teller Opener Platform Requirement,Teller Opener Platform Scheduled,MSR Opener Platform,...,MSR Closer Lobby Scheduled,BM Admin Time,BM Admin Time Requirement,BM Admin Time Scheduled,ABM Admin Time,ABM Admin Time Requirement,ABM Admin Time Scheduled,BMT Meeting,BMT Meeting Requirement,BMT Meeting Scheduled
01:00:00,3.0,3.9,2.0,3.0,3.5,4.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
01:15:00,4.0,5.6,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
01:30:00,4.0,5.3,3.0,2.0,2.9,3.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
01:45:00,4.0,4.8,3.0,2.0,2.4,3.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
02:00:00,3.0,3.8,3.0,2.0,2.2,2.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
02:15:00,4.0,4.9,3.0,2.0,2.1,2.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
02:30:00,4.0,4.7,3.0,2.0,2.1,2.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
02:45:00,4.0,4.3,2.0,2.0,2.1,2.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
03:00:00,2.0,2.1,2.0,2.0,2.5,2.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
03:15:00,3.0,3.0,2.0,3.0,3.0,2.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [55]:
from matplotlib.widgets import Slider, Button
%matplotlib tk

fig = x3.plot.bar(color = ['#228B22', 'blue', '#BB0000', '#228B22', 'orange', '#BB0000', '#228B22', 'purple', '#BB0000',
                      '#228B22', 'yellow', '#BB0000', '#228B22', 'black', '#BB0000', '#228B22', 'pink', '#BB0000',
                      '#228B22', '#804000', '#BB0000', '#228B22', '#86b300', '#BB0000', '#228B22', '#3399ff', '#BB0000'], 
            width=1, stacked=False, align='edge');

# a = ScrollableWindow(fig)
# plt.savefig('analysis.pdf')
# downtownPlatform.plot.bar(color='red', alpha=0.5, stacked=False);
# bmAdminTime.plot.bar(color='blue', alpha=0.5, stacked=False);

In [ ]:
for value, item in Dic:
        print(value, item)